In [1]:
import torch
from omegaconf import OmegaConf
from src.models.blm.config import ModelArgs
from src.models.blm.pl_dataloader import TinyStoriesDataloader
from src.models.blm.pl_training import Transformer
from src.tokenize.tokenizer import Tokenizer

Seed set to 123
Seed set to 123


In [57]:
from src.models.blm.finetune.selected_layers_finetune_causal import FineTuneBLM

In [199]:
from datasets import load_dataset, load_from_disk

ds = load_from_disk(
    "/home/pranav-pc/projects/OpenTransformer/multiformer/data/finetune/instruct-dataset-v3"
)

In [59]:
BASE_URL = "/home/pranav-pc/projects/OpenTransformer/multiformer"
tokenizer_path = BASE_URL + "/tokenizer_checkpoints"
tokenizer = Tokenizer(tokenizer_path)

In [60]:
MODEL_CONFIG_PATH = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/src/models/blm/conf/config.yaml"
)
MODEL_CHECKPOINT_PATH = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/blm-instruct/last-v2.ckpt"
)

conf = OmegaConf.load(MODEL_CONFIG_PATH)

In [61]:
state_dict = torch.load(MODEL_CHECKPOINT_PATH)["state_dict"]
# Pytorch by default add _orig_mod in the checkpoint keys.#TODO: Take care of this while model checkpointing
state_dict = {k.replace("._orig_mod", ""): v for k, v in state_dict.items()}

In [62]:
config = ModelArgs(**conf["model"])
model = FineTuneBLM(Transformer(config))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [63]:
#### Inference
model.eval()
model = model.cuda()

In [200]:
ds = ds.filter(lambda example: example["source"] == "samsum", num_proc=30)

Filter (num_proc=30):   0%|          | 0/4311905 [00:00<?, ? examples/s]

Filter (num_proc=30):   0%|          | 0/1077977 [00:00<?, ? examples/s]

In [202]:
ds["validation"][7]

{'prompt': "Create a revised version of the following passage with improved structure and flow.### Instruction:\nWilson: Ticket guy just nabbed me\r\nKaia: Damn..\r\nWilson: Even though I had a ticket. I am gonna go to report it\r\nKaia: So how come? Did u show him ur ticket and still he gave u a fine? Of course u need to report it\r\nWilson: We got in the same time and he turned of the machine before I could put my ticket\r\nKaia: Don't worry. There are cameras. They will check them and if it was as u say, u won't have to pay anything. Was there anyone who saw it that u tried to put ur ticket but it was too late?\r\nWilson: Huh this was the old tram\r\nKaia: Did u tell him u wanted to put ur ticket but he turned off the machine first?\r\nWilson: No english\r\nKaia: Stupid... I will go to the office with u if u want\r\nWilson: No need\r\nKaia: Ok. Better if u go to the office as fast as possible\r\nWilson: On my way there\n\n### Response:\nWilson is going to the office to report an unj

In [191]:
text = """Identify grammatical mistakes in the following passage and provide corrections.### Instruction:\nLet us try to learn grammer.\n\n### Response:\n"""

In [192]:
print(text)

Identify grammatical mistakes in the following passage and provide corrections.### Instruction:
Let us try to learn grammer.

### Response:



In [193]:
# text = "Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. It was sad because it couldn’t move. Every day, it would say"
# text = "Jack was hungry, so he went looking for"
# text = "Tim is a good boy. one day his father called and asked for the school exam result"
# text = "Jack wanted to read a book,so he went to"
# text = "My name is Mariama, my favorite"
# text = "User: Words: come, road, sad Summary: A bus becomes jealous of a shiny new car and undergoes an operation to go faster, becoming good friends with the car and making everyone in the town happy. Assistant:"
# text = "### Instruction:\nHow can i motivate myself to go to gym?\n\n### Response:\n"
for i in range(3):
    tokens = torch.LongTensor(tokenizer.encode(text)).to("cuda:0").view(1, -1)[:, :-1]
    # print(tokens)
    predicted_tokens = model.predict_step(
        tokens, None, max_new_tokens=1024, temperature=0.9, top_k=5, conditional_break=[2]
    )[0].tolist()
    # print(predicted_tokens)
    print(tokenizer.decode_ids(predicted_tokens))
    print("==" * 30)

To learn, let us learn the basics of learning.

Let us try to learn, let us try to learn a lot, let us learn the basics of learning, let us learn the basics.
Let we try to learn a new language and communicate with others.

To learn a new language, let us try to learn how and communicate with others. Let us communicate with others to communicate with others and communicate.
Let us try to learn how to be able.

The following passage is a list of teaching the instructions that is a good way to learn how to be able to become a good student by learning about the process.


In [73]:
tokenizer.id_to_piece(13)

'<0x0A>'